In [1]:
import requests
import pandas as pd
import json 
payload = {"brand":"GRUBHUB","client_id":"beta_UmWlpstzQSFmocLy3h1UieYcVST","device_id":637099320,"refresh_token":"20be56ef-022d-4086-a791-4367c931f84b"}
s = requests.Session()
str_resp = s.post('https://api-gtm.grubhub.com/auth',json=payload)
json_res = json.loads(str_resp.text)
auth = json_res.get('session_handle').get('access_token')
auth_token = {'authorization': 'Bearer '+ auth}

In [8]:
Category, ItemName, Description,ItemPrice = ([] for i in range(4))
str_resp = requests.get('https://api-gtm.grubhub.com/restaurants/2159864?hideChoiceCategories=true&version=4&variationId=rtpFreeItems&orderType=standard&hideUnavailableMenuItems=true&hideMenuItems=false',headers=auth_token)
json_res = json.loads(str_resp.text)
data= json_res["restaurant"]["menu_category_list"][1]["menu_item_list"]

def remov_curr(rate):
    rate.pop("currency",None)
    return rate['amount']/100.0

   
for each_item in data:
    for i in range(0,7):
        Category.append(each_item["menu_category_name"])
        ItemName.append(each_item["name"])
        Description.append(each_item["description"])
        ItemPrice.append(remov_curr(each_item["price"]))
        
df1 = pd.DataFrame()   
df1 = pd.DataFrame({'Category':Category,'ItemName':ItemName,'Description':Description,'ItemPrice':ItemPrice})

In [7]:
str_resp = requests.get('https://api-gtm.grubhub.com/restaurants/2159864/menu_items/?menuItemIds=1772682880&menuItemIds=1771765220&menuItemIds=1773302171&menuItemIds=1778765332&menuItemIds=1772682372&menuItemIds=1833943302&menuItemIds=1776120866&menuItemIds=1776124734&menuItemIds=1771944571&menuItemIds=1771765739&menuItemIds=1773301444&menuItemIds=1771466470&menuItemIds=1776116192&menuItemIds=1835179712&menuItemIds=1776116193&menuItemIds=1771454024&menuItemIds=1771941372&menuItemIds=1772686226&menuItemIds=1773472174&menuItemIds=1771940438&menuItemIds=1771461879&menuItemIds=1772686476&menuItemIds=1773301432&menuItemIds=1771941358&menuItemIds=1771453490&menuItemIds=1771748524&orderType=standard&version=4',headers=auth_token)
json_res = json.loads(str_resp.text)
Options,OptionPrice = [],[]
data = json_res["menu_items"][2]["choice_category_list"][0]['choice_option_list']
OptionName = json_res["menu_items"][2]["choice_category_list"][0]['name']

def func(rate):
    rate.pop("currency",None)
    return rate['amount']/100.0

for each_item in data:
    Options.append(each_item['description'])
    OptionPrice.append(func(each_item['price']))
df2 = pd.DataFrame({'OptionName':OptionName,'Options':Options*12,'OptionPrice':OptionPrice*12})
df = df1.join(df2)

In [6]:
df.to_csv(r'C:\Users\91789\Desktop\GrubHub_Data.csv',encoding='utf-8',index=False)
